In [1]:
from setting.db import SessionLocal

from graph.graph_knowledge_base import GraphKnowledgeBase

gkb = GraphKnowledgeBase("entities_150001", "relationships_150001", "chunks_150001")
session = SessionLocal()

In [2]:
query = "what is the latest version of tidb"
model_kwargs = {
    "num_ctx": 8092,
    "num_gpu": 40,
    "num_predict": 10000,
    "temperature": 0.1,
}

In [4]:
from llm_inference.base import LLMInterface
from graph.query_analyzer import DeepUnderstandingAnalyzer

llm_client = LLMInterface("openai", "gpt-4o")
analyzer = DeepUnderstandingAnalyzer(llm_client)
analysis_res = analyzer.perform(query)
print(analysis_res)

Analysis Result:
Reasoning: To deeply analyze the query 'what is the latest version of tidb' using first principles thinking, we start by understanding the user's fundamental need. The user is seeking information about the current state of a specific software product, TiDB. This indicates a need for up-to-date information, which is crucial in the context of software development and deployment. The fundamental components needed to answer this question include identifying what TiDB is, understanding how software versions are tracked and released, and determining where the most reliable and current information about TiDB's version can be found. The user might be asking this question for several reasons: they could be considering an upgrade, evaluating compatibility with other systems, or simply staying informed about the software they use. We would know if we've fully answered the question by providing the most recent version number of TiDB, along with a source or method for verifying thi

In [5]:
from graph.knowledge_synthesizer import KnowledgeSynthesizer, SearchAction

synthesizer = KnowledgeSynthesizer(llm_client)

In [6]:
action_history = []
current_findings = []
docs = {}

next_actions = [SearchAction(
    tool="retrieve_knowledge",
    query=a
) for a in analysis_res.initial_queries]

reasoning = analysis_res.reasoning
queries = analysis_res.initial_queries

In [7]:
import datetime

# Fact 1: 
current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
prompt = f"Current Time: {current_time}.\n\nThe question: {query}.\n\nThe Reasoning: {reasoning}.\n\nSo the answer is:"

answer_1 = llm_client.generate(prompt)
answer_1

LLM generation failed: llm_inference.base.BaseLLMProvider._retry_with_exponential_backoff() got multiple values for keyword argument 'temperature'


TypeError: llm_inference.base.BaseLLMProvider._retry_with_exponential_backoff() got multiple values for keyword argument 'temperature'

In [ ]:

knowledge_retrieved = {}
for action in next_actions:
    if action.tool == 'retrieve_knowledge':
        data = gkb.retrieve_graph_data(session, action.query)
    elif action.tool == 'retrieve_neighbors':
        data = gkb.retrieve_neighbors(session, action.entity_ids, action.query)
    knowledge_retrieved[str(action)] = data
    action_history.append(action)

knowledge_retrieved

In [ ]:
current_findings = synthesizer.extract_findings(reasoning, query, knowledge_retrieved, current_findings)
current_findings

In [ ]:
for finding in current_findings:
    for quote in finding.source_quotes:
        refer_chunks = {}
        refer_relationships = {}
        doc_link = quote.get('doc_link')
        if doc_link is None:
            print(f"doc_link is None for quote: {quote}")
        
        for chunk_id in quote.get('chunk_ids', []):
            is_found = False
            for action, data in knowledge_retrieved.items():
                for ck in data.get('chunks', []):
                    if ck.get('id') == chunk_id:
                        refer_chunks[chunk_id] = ck
                        is_found = True
                        break
                if is_found:
                    break

        for relationship_id in quote.get('relationship_ids', []):
            is_found = False
            for action, data in knowledge_retrieved.items():
                for rel in data.get('relationships', []):
                    if rel.get('id') == relationship_id:
                        refer_relationships[relationship_id] = rel
                        is_found = True
                        break
                if is_found:
                    break

        if doc_link not in docs:
            docs[doc_link] = {
                'refer_chunks': refer_chunks,
                'refer_relationships': refer_relationships,
            }
        else:
            docs[doc_link]['refer_chunks'].update(refer_chunks)
            docs[doc_link]['refer_relationships'].update(refer_relationships)

docs

In [ ]:
eval_res = synthesizer.evaluate_findings(query, reasoning, current_findings, docs, action_history)
eval_res

In [ ]:
next_actions = []
if eval_res.is_sufficient is False:
    print(f"Not sufficient, re-synthesizing...\n    Reasoning: {eval_res.reasoning}\n    missing_aspects: {eval_res.missing_aspects}")
    for action in eval_res.next_actions:
        next_actions.append(action)
next_actions